In [ ]:
!pip install pandas openpyxl requests beautifulsoup4

# Extracción de Datos de Web wol.jw.org - Semanas Reunión

In [3]:
# ========================================
# SCRIPT: EXTRACTOR DE INFORMACIÓN WOL - GOOGLE COLAB
# ========================================
# INSTRUCCIONES: Solo necesitas cambiar las URLs de abajo por las semanas que quieres procesar
# Ejemplo de URL: "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/11/3"

URLS_A_PROCESAR = [
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025401",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025402",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025403",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025404",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025405",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025406",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025407",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025408",
    "https://wol.jw.org/es/wol/d/r4/lp-s/202025409"

    # AGREGAR MÁS URLs AQUÍ - UNA POR LÍNEA
    # "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/12/2",
    # "https://wol.jw.org/es/wol/meetings/r4/lp-s/2024/12/3",
]

# ========================================
# INSTALACIÓN DE DEPENDENCIAS (SOLO PARA COLAB)
# ========================================

# Instalar dependencias necesarias en Google Colab
try:
    import google.colab
    print("🔧 Instalando dependencias para Google Colab...")
    !pip install requests beautifulsoup4 pandas openpyxl -q
    print("✅ Dependencias instaladas correctamente")
except ImportError:
    print("ℹ️  Ejecutándose fuera de Google Colab - usando dependencias del sistema")

# ========================================
# CÓDIGO PRINCIPAL
# ========================================

import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

def extraer_informacion(url):
    """
    Extrae información de una página de WOL de reunión de entre semana.

    Parámetros:
    url (str): URL de la página WOL

    Retorna:
    dict: Diccionario con la información extraída
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # Inicializar el diccionario de información
        info = {
            "semana": "",
            "libro": "",
            "cancion_inicial": "",
            "tesoros": "",
            "segunda_cancion": "",
            "tercera_cancion": "",
            "titulos_maestros": [],
            "maestros_con_contenido": [],  # Nueva información detallada
            "nvc_titulo1": "",
            "nvc_titulo2": "",
            "nvc_titulo3": ""
        }

        # 1. EXTRAER SEMANA - Buscar h1 con id='p1'
        semana_elem = soup.find('h1', id='p1')
        if semana_elem:
            info["semana"] = semana_elem.get_text(strip=True)

        # 2. EXTRAER LIBRO DE ESTUDIO - Buscar h2 con id='p2'
        libro_elem = soup.find('h2', id='p2')
        if libro_elem:
            strong_tag = libro_elem.find('strong')
            if strong_tag:
                info["libro"] = strong_tag.get_text(strip=True)
            else:
                info["libro"] = libro_elem.get_text(strip=True)

        # 3. EXTRAER CANCIONES
        # Primera canción - h3 con id='p3'
        cancion_elem = soup.find('h3', id='p3')
        if cancion_elem:
            cancion_text = cancion_elem.get_text(strip=True)
            match = re.search(r'Canción\s+(\d+)', cancion_text)
            if match:
                info["cancion_inicial"] = f"Canción {match.group(1)}"

        # Extraer todas las canciones restantes
        todas_h3 = soup.find_all('h3')
        canciones_encontradas = []
        cancion_inicial_texto = cancion_elem.get_text(strip=True) if cancion_elem else ""

        for h3 in todas_h3:
            texto = h3.get_text(strip=True)
            if "Canción" in texto and texto != cancion_inicial_texto:
                match = re.search(r'Canción\s+(\d+)', texto)
                if match:
                    cancion_formateada = f"Canción {match.group(1)}"
                    if cancion_formateada not in canciones_encontradas:
                        canciones_encontradas.append(cancion_formateada)

        # Asignar segunda y tercera canción
        if len(canciones_encontradas) >= 1:
            info["segunda_cancion"] = canciones_encontradas[0]
        if len(canciones_encontradas) >= 2:
            info["tercera_cancion"] = canciones_encontradas[1]

        # 4. EXTRAER TESOROS DE LA BIBLIA - Buscar h5, h4 o h3 con id='p5'
        tesoros_elem = soup.find(['h5', 'h4', 'h3'], id='p5')
        if tesoros_elem:
            info["tesoros"] = tesoros_elem.get_text(strip=True)

        # 5. EXTRAER TÍTULOS Y CONTENIDO DE "SEAMOS MEJORES MAESTROS"
        # Buscar el encabezado de la sección
        maestros_element = None
        vida_cristiana_element = None

        for h2 in soup.find_all('h2'):
            texto = h2.get_text(strip=True)
            if "SEAMOS MEJORES MAESTROS" in texto:
                maestros_element = h2
            elif "NUESTRA VIDA CRISTIANA" in texto:
                vida_cristiana_element = h2

        # Extraer títulos y contenido entre "SEAMOS MEJORES MAESTROS" y "NUESTRA VIDA CRISTIANA"
        maestros_titulos_con_contenido = []

        if maestros_element and vida_cristiana_element:
            # Método basado en IDs
            maestros_id = maestros_element.get('id', '')
            vida_id = vida_cristiana_element.get('id', '')

            if maestros_id and vida_id:
                maestros_num = int(maestros_id.replace('p', '')) if maestros_id.replace('p', '').isdigit() else 0
                vida_num = int(vida_id.replace('p', '')) if vida_id.replace('p', '').isdigit() else 1000

                # Buscar h3 con IDs entre estos dos valores
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))
                        if maestros_num < h3_num < vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)
                            if not strong_text.startswith("Canción"):
                                # Extraer el título
                                titulo = strong_text

                                # Extraer el contenido - buscar el div siguiente
                                contenido = ""
                                siguiente_elemento = h3.find_next_sibling()

                                # Buscar el div que contiene el contenido
                                if siguiente_elemento and siguiente_elemento.name == 'div':
                                    # Extraer todo el texto del div, limpiando espacios extra
                                    contenido_raw = siguiente_elemento.get_text(strip=True)
                                    # Limpiar el contenido eliminando saltos de línea excesivos
                                    contenido = ' '.join(contenido_raw.split())

                                # Si no encontramos contenido en el div siguiente, buscar en el párrafo siguiente
                                if not contenido:
                                    siguiente_p = h3.find_next('p')
                                    if siguiente_p:
                                        contenido_raw = siguiente_p.get_text(strip=True)
                                        contenido = ' '.join(contenido_raw.split())

                                # Guardar título y contenido
                                maestros_titulos_con_contenido.append({
                                    'titulo': titulo,
                                    'contenido': contenido
                                })

                                # Mantener compatibilidad con el formato anterior
                                info["titulos_maestros"].append(titulo)

                # Guardar la información detallada
                info["maestros_con_contenido"] = maestros_titulos_con_contenido

        # 6. EXTRAER TÍTULOS DE "NUESTRA VIDA CRISTIANA"
        titulos_nvc = []
        estudio_biblico_texto = None

        if vida_cristiana_element:
            vida_id = vida_cristiana_element.get('id', '')
            if vida_id and vida_id.replace('p', '').isdigit():
                vida_num = int(vida_id.replace('p', ''))

                # Buscar todos los h3 con ID mayor que el de "NUESTRA VIDA CRISTIANA"
                for h3 in soup.find_all('h3'):
                    h3_id = h3.get('id', '')
                    if h3_id and h3_id.replace('p', '').isdigit():
                        h3_num = int(h3_id.replace('p', ''))

                        if h3_num > vida_num and h3.find('strong'):
                            strong_text = h3.find('strong').get_text(strip=True)

                            # Si encontramos "Estudio bíblico de la congregación"
                            if "Estudio bíblico de la congregación" in strong_text:
                                estudio_biblico_texto = strong_text
                                break  # Terminamos la búsqueda una vez encontrado

                            # Otros títulos que no son canciones
                            elif not strong_text.startswith("Canción"):
                                titulos_nvc.append(strong_text)

        # Buscar "Estudio bíblico de la congregación" si no se encontró aún
        if not estudio_biblico_texto:
            for h3 in soup.find_all('h3'):
                if h3.find('strong') and "Estudio bíblico de la congregación" in h3.get_text(strip=True):
                    estudio_biblico_texto = h3.find('strong').get_text(strip=True)
                    break

        # Asignar los títulos NVC
        info["nvc_titulo1"] = titulos_nvc[0] if len(titulos_nvc) > 0 else ""
        info["nvc_titulo2"] = titulos_nvc[1] if len(titulos_nvc) > 1 else ""
        info["nvc_titulo3"] = estudio_biblico_texto if estudio_biblico_texto else "Estudio bíblico de la congregación"

        return info

    except Exception as e:
        print(f"Error al extraer información de {url}: {str(e)}")
        return None

def procesar_urls_wol(urls):
    """
    Procesa una lista de URLs de WOL y crea un archivo Excel independiente.

    Parámetros:
    urls (list): Lista de URLs a procesar

    Retorna:
    tuple: (datos_procesados, nombre_archivo)
    """
    datos = []

    # Nombre del archivo con timestamp para evitar conflictos
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"info-reunion_{timestamp}.xlsx"

    print("🔍 Extrayendo información de las páginas WOL...")
    print("-" * 50)

    # Extraer información de cada URL
    for i, url in enumerate(urls, 1):
        try:
            print(f"📄 ({i}/{len(urls)}) Procesando: {url}")
            info = extraer_informacion(url)

            if info is None:
                print("   ❌ No se pudo extraer información")
                continue

            # Información básica para el DataFrame
            datos_basicos = {
                "Semana": info["semana"],
                "Libro": info["libro"],
                "Canción Inicial": info["cancion_inicial"],
                "Tesoros de la Biblia": info["tesoros"],
                "Segunda Canción": info["segunda_cancion"],
                "Tercera Canción": info["tercera_cancion"]
            }

            # Añadir los títulos de maestros como columnas separadas (solo títulos)
            for j, titulo in enumerate(info["titulos_maestros"], 1):
                if j <= 4:  # Máximo 4 títulos
                    datos_basicos[f"Maestros Título {j}"] = titulo

            # Rellenar títulos de maestros faltantes con cadena vacía
            for j in range(len(info["titulos_maestros"]) + 1, 5):
                datos_basicos[f"Maestros Título {j}"] = ""

            # Añadir el contenido de maestros como columnas separadas
            for j, item in enumerate(info["maestros_con_contenido"], 1):
                if j <= 4:  # Máximo 4 contenidos
                    datos_basicos[f"Maestros Contenido {j}"] = item['contenido']

            # Rellenar contenidos de maestros faltantes con cadena vacía
            for j in range(len(info["maestros_con_contenido"]) + 1, 5):
                datos_basicos[f"Maestros Contenido {j}"] = ""

            # Añadir los títulos NVC
            datos_basicos["NVC Título 1"] = info["nvc_titulo1"]
            datos_basicos["NVC Título 2"] = info["nvc_titulo2"]
            datos_basicos["NVC Título 3"] = info["nvc_titulo3"]

            datos.append(datos_basicos)
            print(f"   ✅ Semana extraída: {info['semana']}")

            # Mostrar información detallada para verificación
            print("   📋 Información extraída:")
            print(f"      • Libro: {info['libro']}")
            print(f"      • Canciones: {info['cancion_inicial']} | {info['segunda_cancion']} | {info['tercera_cancion']}")
            if info['tesoros']:
                tesoros_corto = info['tesoros'][:50] + "..." if len(info['tesoros']) > 50 else info['tesoros']
                print(f"      • Tesoros: {tesoros_corto}")
            print(f"      • Maestros ({len(info['titulos_maestros'])}): {', '.join(info['titulos_maestros'])}")

            # Mostrar contenido detallado de maestros
            if info['maestros_con_contenido']:
                print("      • Contenido Maestros:")
                for i, item in enumerate(info['maestros_con_contenido'], 1):
                    contenido_corto = item['contenido'][:60] + "..." if len(item['contenido']) > 60 else item['contenido']
                    print(f"        {i}. {item['titulo']}: {contenido_corto}")

            print(f"      • NVC: {info['nvc_titulo1']} | {info['nvc_titulo2']} | {info['nvc_titulo3']}")
            print()

        except Exception as e:
            print(f"   ❌ Error al procesar: {str(e)}")

        time.sleep(1)  # Pausa entre solicitudes

    print("-" * 50)

    # Crear archivo Excel independiente
    if datos:
        try:
            df = pd.DataFrame(datos)

            print(f"💾 Creando archivo Excel: {nombre_archivo}")

            # Crear el archivo Excel con una sola hoja
            with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:
                df.to_excel(writer, sheet_name='Info-reunión', index=False)

            print(f"   ✅ Archivo creado exitosamente: {nombre_archivo}")
            return datos, nombre_archivo

        except Exception as e:
            print(f"   ❌ Error al crear el archivo Excel: {str(e)}")
            return None, None
    else:
        print("   ⚠️  No hay datos para guardar")
        return None, None

def descargar_archivo_colab(nombre_archivo):
    """
    Función para descargar el archivo en Google Colab

    Parámetros:
    nombre_archivo (str): Nombre del archivo a descargar
    """
    import os

    # Verificar que el archivo existe
    if not os.path.exists(nombre_archivo):
        print(f"❌ No se encontró el archivo: {nombre_archivo}")
        return False

    # Mostrar información del archivo
    size = os.path.getsize(nombre_archivo)
    print(f"📄 Archivo encontrado: {nombre_archivo}")
    print(f"💾 Tamaño: {size} bytes")
    print(f"📍 Ruta completa: {os.path.abspath(nombre_archivo)}")

    try:
        from google.colab import files
        print(f"📥 Iniciando descarga del archivo: {nombre_archivo}")
        files.download(nombre_archivo)
        print("✅ Descarga iniciada - revisa la carpeta de descargas de tu navegador")
        return True
    except ImportError:
        print("ℹ️  No estás en Google Colab - el archivo está guardado en:")
        print(f"   {os.path.abspath(nombre_archivo)}")
        return True
    except Exception as e:
        print(f"❌ Error al descargar: {str(e)}")
        print("\n🔧 SOLUCIÓN ALTERNATIVA:")
        print("1. Ve al panel de archivos en el lado izquierdo de Colab (ícono de carpeta)")
        print(f"2. Busca el archivo: {nombre_archivo}")
        print("3. Haz clic derecho → Descargar")
        return False

# ========================================
# EJECUCIÓN AUTOMÁTICA
# ========================================
if __name__ == "__main__":
    print("="*60)
    print("🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC (GOOGLE COLAB)")
    print("="*60)
    print(f"📋 Se procesarán {len(URLS_A_PROCESAR)} páginas...")
    print()

    # Procesar las URLs y extraer información
    datos_procesados, archivo_creado = procesar_urls_wol(URLS_A_PROCESAR)

    if datos_procesados and archivo_creado:
        print("\n" + "="*60)
        print("✅ INFORMACIÓN EXTRAÍDA CORRECTAMENTE")
        print("="*60)
        print(f"📊 Total de semanas procesadas: {len(datos_procesados)}")
        print(f"📁 Archivo creado: {archivo_creado}")
        print(f"📋 Hoja de datos: 'Info-reunión'")

        # Intentar descargar automáticamente en Google Colab
        print("\n🔍 DESCARGA DEL ARCHIVO:")
        descarga_exitosa = descargar_archivo_colab(archivo_creado)

        if not descarga_exitosa:
            print("\n📋 INSTRUCCIONES PARA DESCARGAR MANUALMENTE:")
            print("1. Ve al panel izquierdo de Google Colab")
            print("2. Haz clic en el ícono de carpeta 📁")
            print(f"3. Busca el archivo: {archivo_creado}")
            print("4. Haz clic derecho sobre el archivo")
            print("5. Selecciona 'Descargar'")

        print("\n🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!")

        # Mostrar resumen de los datos
        print("\n📋 RESUMEN DE DATOS EXTRAÍDOS:")
        for i, dato in enumerate(datos_procesados, 1):
            print(f"   {i}. {dato['Semana']}")

        # Comando adicional para listar archivos
        print(f"\n🔧 COMANDO PARA DESCARGAR MANUALMENTE:")
        print(f"files.download('{archivo_creado}')")

    else:
        print("\n❌ No se pudo procesar ninguna URL o crear el archivo")

    # Mostrar todos los archivos info-reunion en el directorio
    import os
    archivos_info = [f for f in os.listdir('.') if f.startswith('info-reunion')]
    if archivos_info:
        print(f"\n📁 ARCHIVOS 'info-reunion' ENCONTRADOS:")
        for archivo in archivos_info:
            size = os.path.getsize(archivo)
            print(f"   📄 {archivo} ({size} bytes)")

    print("\n" + "="*60)

🔧 Instalando dependencias para Google Colab...
✅ Dependencias instaladas correctamente
🔄 EXTRACTOR DE INFORMACIÓN WOL - VMC (GOOGLE COLAB)
📋 Se procesarán 9 páginas...

🔍 Extrayendo información de las páginas WOL...
--------------------------------------------------
📄 (1/9) Procesando: https://wol.jw.org/es/wol/d/r4/lp-s/202025401
   ✅ Semana extraída: 3-9 DE NOVIEMBRE
   📋 Información extraída:
      • Libro: EL CANTAR DE LOS CANTARES 1,
      • Canciones: Canción 132 | Canción 46 | Canción 137
      • Tesoros: 1. Una historia de amor verdadero
      • Maestros (3): 4. Empiece conversaciones, 5. Haga revisitas, 6. Haga discípulos
      • Contenido Maestros:
        1. 4. Empiece conversaciones: (3 mins.) DE CASA EN CASA. Utilice una de las verdades delap...
        2. 5. Haga revisitas: (4 mins.) DE CASA EN CASA. Utilice una de las verdades delap...
        3. 6. Haga discípulos: (5 mins.)lfflección 18 introducción y puntos 1-3(thlección 8...
      • NVC: 7. “El generoso será bendecid

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga iniciada - revisa la carpeta de descargas de tu navegador

🎉 ¡EXTRACCIÓN COMPLETADA EXITOSAMENTE!

📋 RESUMEN DE DATOS EXTRAÍDOS:
   1. 3-9 DE NOVIEMBRE
   2. 10-16 DE NOVIEMBRE
   3. 17-23 DE NOVIEMBRE
   4. 24-30 DE NOVIEMBRE
   5. 1-7 DE DICIEMBRE
   6. 8-14 DE DICIEMBRE
   7. 15-21 DE DICIEMBRE
   8. 22-28 DE DICIEMBRE
   9. 29 DE DICIEMBRE DE 2025 A 4 DE ENERO DE 2026

🔧 COMANDO PARA DESCARGAR MANUALMENTE:
files.download('info-reunion_20251027_162711.xlsx')

📁 ARCHIVOS 'info-reunion' ENCONTRADOS:
   📄 info-reunion_20251027_161514.xlsx (5516 bytes)
   📄 info-reunion_20251027_162109.xlsx (5822 bytes)
   📄 info-reunion_20251027_162711.xlsx (7559 bytes)

